# 1. Cellpose Segmentation

In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.getcwd()))

In [2]:
from src.segmentation import CellposeSegmenter

In [3]:
# Initialize the CellposeSegmenter
segmenter = CellposeSegmenter(
    gpu=True,  # Set to False if you don't have GPU
    model_type='nuclei',
    diam_mean=19.0
)

Initializing denoise model...
Denoise model initialized
Initializing cellpose model...
Cellpose model initialized


In [4]:
# Check files in your folders
# Make sure to use relative paths from where your notebook is
results = segmenter.check_files_in_folder(
    input_folder="data/input",
    output_folder="data/output"
)

Checking input folder: data/input
Contents of input folder data/input:
- cropped_polygon_x3000_y12000_width800.tif
- cropped_polygon_x7247_y14117_width800.tif


In [ ]:
segmenter.process_images_in_folder("data/input", "data/output")